In [8]:
import pandas as pd
import os
from datetime import timedelta
import matplotlib.pyplot as plt

In [48]:
def converter(s):
    parts = s.split()
    integer_part = int(parts[0])
    fraction_part = eval(parts[1])
    result = integer_part + fraction_part
    return result

def corr_finder(weatherdf,pricedf):
    weatherdf["Date"] = pd.to_datetime(weatherdf["Date"])
    pricedf["Date"] = pd.to_datetime(pricedf["Date"])
    df = pd.merge(weatherdf,pricedf,on='Date',how='inner')

    temp_col_name = df.columns[1]
    price_col_name = df.columns[2]

    df[price_col_name] = df[price_col_name].apply(lambda x: converter(x) if ' ' in str(x) else x)
    correlation = df[temp_col_name].corr(df[price_col_name].diff())
    # plt.scatter(df["Last Price_x"], df["Last Price_y"])
    return correlation

    # df = pd.merge(weatherdf,pricedf,on='Date',how='inner').iloc[:,[0,1,-1]]
    # print(df)
    # correlation = df.iloc[:,1].corr(df.iloc[:,2])
    # plt.scatter(df.iloc[:,1], df.iloc[:,2])
    # return correlation

In [30]:
station_names = os.listdir(os.getcwd() + "/36 month Degree day")
commoditynames = pd.ExcelFile('commodity_prices.xlsx').sheet_names
names = [['New York', 'KJFK_Temps_Last36.csv'], ['Chicago', 'KORD_Temps_Last36.csv'], ['Boston', 'KBOS_Temps_Last36.csv'],['Philidelphia', 'KPHL_Temps_Last36.csv'],['Denver', 'KDEN_Temps_Last36.csv'],['Atlanta', 'KATL_Temps_Last36.csv'],['Minneapolis', 'KMSP_Temps_Last36.csv'],['Phoenix', 'KPHX_Temps_Last36.csv'],['Portland, ME', 'KPWM_Temps_Last36.csv'],['Houston', 'KIAH_Temps_Last36.csv'],['Detroit', 'KDTW_Temps_Last36.csv']]
names = {b:a for a,b in names}
matrix_station = pd.DataFrame(index=names.values(),columns=commoditynames)

In [31]:
for station in station_names:
    for commodity in commoditynames:
        matrix_station.loc[names[station], commodity] = corr_finder(pd.read_csv(f'36 month Degree day/{station}'),pd.read_excel('commodity_prices.xlsx',sheet_name=commodity))

In [27]:
citynames = pd.ExcelFile('bbg_weather.xlsx').sheet_names
commoditynames = pd.ExcelFile('commodity_prices.xlsx').sheet_names
matrix_city = pd.DataFrame(index=citynames,columns=commoditynames)

In [28]:
for city in citynames:
    for commodity in commoditynames:
        matrix_city.loc[city,commodity] = corr_finder(pd.read_excel('bbg_weather.xlsx',sheet_name=city),pd.read_excel('commodity_prices.xlsx',sheet_name=commodity))

In [49]:

# % Estimated
iowa = pd.read_csv('36 month Degree day/KDSM_CDD_50F.csv')
iowa = iowa.drop([iowa.columns[2]], axis=1)
iowa["Date"] = pd.to_datetime(iowa["Date"])
iowa["Month"] = iowa["Date"].dt.month
iowa = iowa.loc[iowa["Month"].isin([5,6,7,8,9])]
iowa = iowa.drop(["Month"], axis=1)

for commodity in commoditynames:
    corr = corr_finder(iowa,pd.read_excel('commodity_prices.xlsx',sheet_name=commodity))
    print(commodity, corr)


NG1 (Natural Gas) -0.09937702181686227
C01 (Brent) -0.15228581604821578
CL1 (WTI) -0.172544452324072
C 1 (Corn) -0.12599138297141144
S 1 (Soy) -0.14441878700779198
W 1 (Wheat) -0.14311010024464849
FC1 (Feeder Cattle) -0.1310403135911767
LC1 (Live Cattle) -0.12561366162455312


In [29]:
matrix_city

,NG1 (Natural Gas),C01 (Brent),CL1 (WTI),C 1 (Corn),S 1 (Soy),W 1 (Wheat),FC1 (Feeder Cattle),LC1 (Live Cattle)
NYC,-0.037744,0.024856,0.011639,0.0578,0.053418,0.023487,-0.020134,0.013541
Chicago,-0.034467,0.004848,-0.006304,0.042324,0.043763,0.008678,-0.030053,0.002301
Dallas,-0.039551,-0.005195,-0.010446,0.055561,0.053676,0.011182,-0.02814,0.023937
Boston,-0.032167,0.029958,0.019322,0.056536,0.056177,0.035093,-0.024564,0.007827
Iowa,-0.029086,-0.000781,-0.006577,0.051799,0.053998,0.014493,-0.021004,0.013243
Charlotte,-0.037386,0.006079,0.005796,0.042347,0.048053,0.019575,-0.018196,0.023287
LA,-0.01085,0.011378,0.005266,0.036375,0.053523,-0.003609,-0.009833,-0.005681
Miami,-0.031503,0.03817,0.021359,0.042866,0.042326,0.034049,-0.015195,0.024023
Seattle,-0.034069,0.023905,0.018351,0.022798,0.025723,0.000865,-0.01476,0.020959
Denver,-0.002868,0.010831,-0.011522,0.069526,0.076735,0.015781,-0.03123,0.004333


In [32]:
matrix_station

,NG1 (Natural Gas),C01 (Brent),CL1 (WTI),C 1 (Corn),S 1 (Soy),W 1 (Wheat),FC1 (Feeder Cattle),LC1 (Live Cattle)
New York,0.023847,-0.014235,-0.015703,-0.063591,-0.061522,-0.009351,0.025423,0.032948
Chicago,0.035704,-0.006501,-0.010825,-0.066745,-0.071137,-0.021431,0.02381,0.017891
Boston,0.016045,-0.023272,-0.021297,-0.056655,-0.064472,-0.015932,0.018026,0.022086
Philidelphia,0.028127,-0.016145,-0.014022,-0.058852,-0.061207,-0.0046,0.030641,0.029162
Denver,0.020812,-0.03133,-0.013011,-0.056984,-0.098287,0.027033,0.03487,0.023434
Atlanta,0.039081,-0.010143,-0.004267,-0.084267,-0.081624,0.005316,0.036469,0.010475
Minneapolis,0.040319,-0.010984,-0.011002,-0.080233,-0.107942,-0.036391,0.027723,0.017447
Phoenix,0.026854,-0.021124,-0.013867,-0.070891,-0.095504,-0.024839,0.05602,0.0266
"Portland, ME",0.025511,-0.016575,-0.022985,-0.047315,-0.06071,-0.016017,0.019751,0.025228
Houston,0.056843,-0.022883,0.002022,-0.066666,-0.067915,-0.019232,0.042565,0.022231
